In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
pending_day= 14
pending_day = spark.sql(f"""SELECT CASE
    WHEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN DATE '2025-05-08' AND DATE '2025-05-16' THEN {pending_day} + 3
    ELSE {pending_day}
  END AS pending_day""").collect()[0]['pending_day']


In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_winare_dc_open_order_dc
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 AS
with supra AS (
    SELECT DISTINCT dc_id 
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive 
    WHERE supra = 'YES'
      AND replenishment_mode = 'BS'
      AND dc_id = '0000'
),

open_order_sft AS (
    SELECT
        a.dc_id,
        'YES' AS supra,
        a.product_id,
        NULL AS po_pending_day,
        SUM(a.open_order) AS open_order
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sft a
    WHERE a.calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND a.dc_id IN (SELECT dc_id FROM supra)
    GROUP BY 1,2,3,4
),

open_order_sap AS (
    SELECT 
        a.store_id AS dc_id,
        'NO' AS supra,
        a.product_id,
        {pending_day} AS po_pending_day,
        IFNULL(SUM(a.po_qty * c.numerator / c.denomintr), 0) AS open_order
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c 
        ON a.product_id = c.material 
       AND a.po_unit = c.mat_unit
    WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), {pending_day}) 
                              AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
      AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
      AND a.po_gr_date IS NULL
      AND a.gr_date IS NULL
      AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng') OR a.webportal_status IS NULL)
      AND a.po_type_id = 'NB'
      AND a.store_id NOT IN (SELECT dc_id FROM supra)
    GROUP BY 1,2,3,4
),
open_order_union AS (
    SELECT dc_id, supra, product_id, po_pending_day, open_order FROM open_order_sft
    UNION ALL
    SELECT dc_id, supra, product_id, po_pending_day, open_order FROM open_order_sap
)

SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
    a.dc_id,
    a.supra,
    a.product_id,
    a.po_pending_day,
    a.open_order
FROM open_order_union a
""")
